# 신조어 추출 및 코퍼스 생성 테스트

이 노트북은 공공 데이터(SNS, AIHub)에서 신조어를 추출하고 코퍼스를 생성하는 전체 파이프라인을 테스트합니다.

## 목차
1. 환경 설정
2. 데이터 수집
3. 신조어 추출
4. 코퍼스 생성
5. 결과 분석 및 시각화

## 1. 환경 설정

In [ ]:
import sys
sys.path.append('../src')

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# 한글 폰트 설정
plt.rcParams['font.family'] = 'DejaVu Sans'
plt.rcParams['axes.unicode_minus'] = False

print("환경 설정 완료!")

In [ ]:
# 우리 패키지 임포트
from neologism_extractor import NeologismExtractor, DataCollector, CorpusBuilder
from neologism_extractor.data_collector import TwitterCollector, AIHubCollector

print("패키지 임포트 완료!")

## 2. 데이터 수집

다양한 소스에서 텍스트 데이터를 수집합니다.

In [ ]:
# 데이터 수집기 초기화
collector = DataCollector()

# Twitter 수집기 등록 (샘플 데이터 사용)
twitter_collector = TwitterCollector()
collector.register_collector('twitter', twitter_collector)

# 데이터 수집
print("데이터 수집 중...")
all_texts = collector.collect_and_merge(max_results=100)

print(f"\n총 {len(all_texts)}개의 텍스트 수집 완료!")
print("\n샘플 데이터:")
for i, text in enumerate(all_texts[:5], 1):
    print(f"{i}. {text}")

### 샘플 데이터 추가 (테스트용)

실제 API가 없을 경우를 대비해 샘플 데이터를 추가합니다.

In [ ]:
# 샘플 한국어 텍스트 (신조어 포함)
sample_texts = [
    "오늘 완전 꿀잼이었어 ㅋㅋㅋ",
    "점메추 좀 해주세요",
    "갓생 살고 싶다 진짜",
    "이거 레알 대박이네요",
    "오늘 TMI 하나 풀자면 말이야",
    "완전 핵인싸네 ㄷㄷ",
    "JMT 맛집 발견했어요!",
    "오늘 할일 다 끝! 갓생이다",
    "이번주 불금이다 와우",
    "월급루팡 중입니다",
    "오늘 회사에서 칼퇴했어요",
    "주말에 홈캉스 어때?",
    "완전 워라밸 좋은 회사",
    "점메추 진짜 어려워요",
    "꾸꾸까까 하고 싶다",
    "이거 실화냐 레전드네",
    "갓생 루틴 시작!",
    "오늘 점심 뭐먹지 점메추",
    "주말에 뭐하지 주말추천",
    "불금에 치맥 어때요?",
    "갓벽한 하루였다",
    "완전 취저입니다",
    "이건 찐이다 진짜",
    "오늘 완전 점메추 실패",
    "주말에 숲캉스 가고 싶어",
    "일코 지키기 힘들다",
    "오늘도 갓생 살기",
    "완전 꿀잼 영화였어",
    "너무 핵노잼이야",
    "오늘 저녁은 혼밥",
    "완전 대박이네요 이거",
    "레알 찐이에요",
    "갓생 루틴 추천해주세요",
    "오늘은 완전 점심 고민",
    "불금에 치맥 최고"
]

all_texts.extend(sample_texts * 5)  # 빈도를 높이기 위해 5번 반복

print(f"샘플 데이터 추가 후 총 {len(all_texts)}개의 텍스트")

## 3. 신조어 추출

수집된 텍스트에서 신조어를 추출합니다.

In [ ]:
# 신조어 추출기 초기화
extractor = NeologismExtractor(
    min_count=3,           # 최소 3번 이상 출현
    min_cohesion=0.03,     # 낮은 응집도 (더 많은 후보)
    min_length=2,          # 최소 2글자
    max_length=10          # 최대 10글자
)

print("신조어 추출 중...")
neologisms = extractor.extract_neologisms(all_texts)

print(f"\n총 {len(neologisms)}개의 신조어 후보 추출 완료!")

In [ ]:
# 상위 신조어 확인
print("\n=== 상위 20개 신조어 ===")
print(f"{'순위':<5} {'신조어':<15} {'점수':<10} {'빈도':<10} {'응집도':<10}")
print("-" * 60)

for i, (word, stats) in enumerate(list(neologisms.items())[:20], 1):
    print(f"{i:<5} {word:<15} {stats['score']:<10.4f} {stats['count']:<10} {stats['cohesion']:<10.4f}")

In [ ]:
# 문맥과 함께 신조어 추출
print("\n문맥 정보 추출 중...")
neologisms_with_context = extractor.extract_with_context(all_texts, top_n=50)

print("\n=== 신조어 예시 (문맥 포함) ===")
for item in neologisms_with_context[:10]:
    print(f"\n단어: {item['word']}")
    print(f"점수: {item['score']:.4f} | 빈도: {item['count']}")
    print("예시:")
    for example in item['examples'][:2]:
        print(f"  - {example}")

## 4. 코퍼스 생성

추출된 신조어로 코퍼스(사전)를 생성합니다.

In [ ]:
# 코퍼스 빌더 초기화
corpus_builder = CorpusBuilder(output_dir="../corpus")

# 신조어 유형 분류 추가
for word, stats in neologisms.items():
    stats['type'] = extractor.classify_neologism_type(word)

# 사전 생성
metadata = {
    'source': 'Twitter, AIHub',
    'collection_date': datetime.now().isoformat(),
    'total_texts': len(all_texts)
}

dictionary = corpus_builder.build_dictionary(neologisms, metadata=metadata)

print(f"사전 생성 완료!")
print(f"총 단어 수: {dictionary['total_words']}")
print(f"생성 일시: {dictionary['created_at']}")

In [ ]:
# 모든 형식으로 저장
saved_files = corpus_builder.save_all_formats(dictionary, base_filename="neologism_dict")

print("\n저장된 파일:")
for format_type, path in saved_files.items():
    print(f"  {format_type.upper()}: {path}")

## 5. 결과 분석 및 시각화

In [ ]:
# DataFrame으로 변환
df = pd.DataFrame(dictionary['words'])

print("데이터 요약:")
print(df.head(10))

In [ ]:
# 통계 정보
print("\n=== 통계 정보 ===")
print(df[['score', 'frequency', 'cohesion']].describe())

In [ ]:
# 신조어 유형별 분포
print("\n=== 신조어 유형별 분포 ===")
type_counts = df['type'].value_counts()
print(type_counts)

In [ ]:
# 시각화 1: 상위 신조어 빈도
plt.figure(figsize=(12, 6))
top_words = df.head(20)
plt.barh(range(len(top_words)), top_words['frequency'])
plt.yticks(range(len(top_words)), top_words['word'])
plt.xlabel('Frequency')
plt.title('Top 20 Neologisms by Frequency')
plt.gca().invert_yaxis()
plt.tight_layout()
plt.show()

In [ ]:
# 시각화 2: 점수 vs 빈도
plt.figure(figsize=(10, 6))
plt.scatter(df['frequency'], df['score'], alpha=0.5)
plt.xlabel('Frequency')
plt.ylabel('Score')
plt.title('Neologism Score vs Frequency')
plt.xscale('log')
plt.tight_layout()
plt.show()

In [ ]:
# 시각화 3: 유형별 분포
if len(type_counts) > 0:
    plt.figure(figsize=(8, 6))
    type_counts.plot(kind='pie', autopct='%1.1f%%')
    plt.title('Neologism Type Distribution')
    plt.ylabel('')
    plt.tight_layout()
    plt.show()

## 6. S3 업로드 (선택사항)

생성된 코퍼스를 S3에 업로드합니다.

In [ ]:
# S3 업로드 설정 (필요시)
S3_BUCKET = "your-bucket-name"  # 버킷 이름 변경
S3_PREFIX = "neologism-corpus/"

# 업로드 여부
UPLOAD_TO_S3 = False  # True로 변경하면 업로드 실행

if UPLOAD_TO_S3:
    for format_type, local_path in saved_files.items():
        s3_key = f"{S3_PREFIX}{datetime.now().strftime('%Y%m%d')}/{local_path.split('/')[-1]}"
        try:
            corpus_builder.upload_to_s3(local_path, S3_BUCKET, s3_key)
        except Exception as e:
            print(f"업로드 실패 ({format_type}): {e}")
else:
    print("S3 업로드 비활성화됨")

## 완료!

신조어 추출 및 코퍼스 생성이 완료되었습니다.

생성된 파일:
- JSON: 전체 메타데이터 포함
- CSV: 엑셀에서 열람 가능
- TXT: 단순 단어 리스트

다음 단계:
1. AWS Glue Job으로 자동화
2. MWAA로 스케줄링
3. 검색 엔진에 통합